In [52]:
import os, sys, email
import re
import numpy as np 
import pandas as pd
import re


In [48]:
emails_df = pd.read_csv('/Users/peter/Library/CloudStorage/OneDrive-Personal/Documents/MDS_UBC/DATA_533/emails.csv')
print(emails_df.shape)
emails_df.head(5)

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
def get_text_from_email(msg):
    """Extract all text/plain parts of an email."""
    parts = []

    for part in msg.walk():
        # Skip container parts like multipart/alternative
        if part.get_content_maintype() == "multipart":
            continue

        # Skip attachments
        if part.get_content_disposition() == "attachment":
            continue

        # Keep only plain text
        if part.get_content_type() != "text/plain":
            continue

        payload = part.get_payload(decode=True)
        
        if payload is None:
            continue

        charset = part.get_content_charset() or "utf-8"

        try:
            text = payload.decode(charset, errors="replace")
        except (LookupError, AttributeError):
            text = payload.decode("utf-8", errors="replace")

        parts.append(text)

    return "".join(parts).strip()

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs


def strip_reply_chain(text):
    """
    Remove quoted/reply chain segments.
    Keeps only the newest part of the email.
    Handles indented / quoted From/To/Cc lines.
    """

    if not isinstance(text, str):
        return text

    patterns = [
        # Common separators
        r"^-{2,}\s*Original Message\s*-{2,}\s*$",
        r"^-{2,}\s*Forwarded Message\s*-{2,}\s*$",
        r"^-{2,}\s*Forwarded to.*-{2,}\s*$"

        # Gmail / generic style
        r"^[>\s]*On .+wrote:\s*$",

        # Header-style blocks (allow spaces / '>' in front)
        r"^[>\s]*From:\s+.+@.+$",
        r"^[>\s]*To:\s+.*@.*$",
        r"^[>\s]*Cc:\s+.*@.*$",
    ]

    splitter = re.compile("|".join(patterns),
                          flags=re.IGNORECASE | re.MULTILINE)

    match = splitter.search(text)
    if match:
        return text[:match.start()].strip()

    return text.strip()

In [50]:
s_email_df = emails_df.sample(10000, random_state=42)

In [51]:
messages = list(map(email.message_from_string, s_email_df['message']))
s_email_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    s_email_df[key] = [doc[key] for doc in messages]





# Parse content from emails
s_email_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
s_email_df['From'] = s_email_df['From'].map(split_email_addresses)
s_email_df['To'] = s_email_df['To'].map(split_email_addresses)


# Extract the root of 'file' as 'user'
s_email_df['user'] = s_email_df['file'].map(lambda x:x.split('/')[0])
del messages

s_email_df.head(1)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
427616,shackleton-s/sent/1912.,<21013688.1075844564560.JavaMail.evans@thyme>,"Tue, 29 Aug 2000 01:26:00 -0700 (PDT)",(sara.shackleton@enron.com),(william.bradford@enron.com),Re: Credit Derivatives,1.0,text/plain; charset=us-ascii,7bit,Sara Shackleton,William S Bradford,,,\Sara_Shackleton_Dec2000_June2001_1\Notes Fold...,SHACKLETON-S,sshackle.nsf,Bill: Thanks for the info. I also spoke wit...,shackleton-s


In [ ]:
s_email_df["clean_content"] = s_email_df["content"].apply(strip_reply_chain)
subset_df = s_email_df[["From", "To", "Subject", "Date", "content", "clean_content"]]

subset_df.head()



,From,To,Subject,Date,content,clean_content
427616,(sara.shackleton@enron.com),(william.bradford@enron.com),Re: Credit Derivatives,"Tue, 29 Aug 2000 01:26:00 -0700 (PDT)",Bill: Thanks for the info. I also spoke wit...,Bill: Thanks for the info. I also spoke wit...
108773,(pat.clynes@enron.com),(aimee.lannou@enron.com),Meter #1591 Lamay Gaslift,"Mon, 24 Apr 2000 05:43:00 -0700 (PDT)","Aimee,\nPlease check meter #1591 Lamay gas lif...","Aimee,\nPlease check meter #1591 Lamay gas lif..."
355471,(knipe3@msn.com),"(fenner.chet@enron.com, constantine.brian@enro...",Re: man night again?,"Thu, 2 May 2002 04:54:27 -0700 (PDT)",GCCA Crawfish and rip-off raffle & over-priced...,GCCA Crawfish and rip-off raffle & over-priced...
457837,(kalmeida@caiso.com),(chris.stokley@enron.com),"Enron 480, 1480 charges","Wed, 8 Aug 2001 14:35:08 -0700 (PDT)","<<Keoni.zip>> Chris, per your request here ar...","<<Keoni.zip>> Chris, per your request here are..."
124910,(chris.germany@enron.com),(thomas.engel@enron.com),Transport Deal,"Wed, 21 Jun 2000 04:58:00 -0700 (PDT)",I'm trying to change the Receipt Meter on deal...,I'm trying to change the Receipt Meter on deal...
403283,(susan.scott@enron.com),(david.foti@enron.com),Re: WT-1 Electric Service Agreement,"Wed, 10 May 2000 11:22:00 -0700 (PDT)",What if we replace Section 2 with something li...,What if we replace Section 2 with something li...
293966,(phillip.love@enron.com),(delma.salazar@enron.com),0003-0001 OAs as of 4/2000 GL,"Wed, 7 Jun 2000 11:30:00 -0700 (PDT)",---------------------- Forwarded by Phillip M ...,---------------------- Forwarded by Phillip M ...
478830,(cynthia.harkness@enron.com),(mark.taylor@enron.com),Leona Tan,"Thu, 13 Apr 2000 02:42:00 -0700 (PDT)","Dear Mark,\n\nAs per our discussion at the law...","Dear Mark,\n\nAs per our discussion at the law..."
295428,(m..love@enron.com),(stevebonilla@yahoo.com),RE: Hello,"Fri, 9 Nov 2001 13:36:17 -0800 (PST)",got your message last night. What is up? Bet...,got your message last night. What is up? Bet...
137822,(orderdetails@buy.com),(dgiron@enron.com),Your buy.com order is on its way,"Mon, 15 Oct 2001 15:11:42 -0700 (PDT)","\nHello Darron,\n\nJust wanted to let you know...","Hello Darron,\n\nJust wanted to let you know t..."


In [ ]:
#removing rows with subjects that are empty, or empty replies
subset_df['Subject'] = subset_df['Subject'].astype(str).str.strip()
subset_df = subset_df[subset_df['Subject'] != ""]
subset_df = subset_df[subset_df['Subject'].str.upper() != "RE:"]

/var/folders/01/hxm6ksm17lg91kg9ct00ty580000gn/T/ipykernel_79561/2935866792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['Subject'] = subset_df['Subject'].astype(str).str.strip()


In [35]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

texts = subset_df["clean_content"].astype(str).tolist()

embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  
)


print(embeddings.shape)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

(10000, 384)


In [36]:
from sklearn.decomposition import PCA

# embeddings: (n_samples, d)
pca = PCA(n_components=50, random_state=42)
X = pca.fit_transform(embeddings)

import hdbscan
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=40,
    metric='euclidean'
)
labels = clusterer.fit_predict(X)

import numpy as np

# Count clusters (excluding noise = -1)
unique, counts = np.unique(labels[labels != -1], return_counts=True)

# Combine into list of (cluster_id, size)
cluster_sizes = list(zip(unique, counts))

# Sort by size descending
cluster_sizes_sorted = sorted(cluster_sizes, key=lambda x: -x[1])

# Print
print("Cluster sizes (largest → smallest):")
for cid, size in cluster_sizes_sorted:
    print(f"Cluster {cid}: {size} points")


/opt/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/miniconda3/envs/ml_env/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Cluster sizes (largest → smallest):
Cluster 3: 650 points
Cluster 0: 172 points
Cluster 2: 129 points
Cluster 1: 56 points


In [38]:
from sklearn.metrics import silhouette_score


subset_df["cluster"] = labels


mask = labels != -1
if mask.sum() > 1 and len(set(labels[mask])) > 1:
    sil = silhouette_score(X[mask], labels[mask])
    print("Silhouette (no noise):", sil)
else:
    print("Not enough clustered points for silhouette.")

Silhouette (no noise): 0.3987036347389221


/var/folders/01/hxm6ksm17lg91kg9ct00ty580000gn/T/ipykernel_82747/1528182328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["cluster"] = labels


In [46]:
subjects = subset_df[subset_df['cluster'] ==3 ]['clean_content']

for i, s in enumerate(subjects, 1):
    print(f"{i}. {s}")

1. ---------------------- Forwarded by Phillip M Love/HOU/ECT on 06/07/2000 
06:26 PM ---------------------------


Jim Little
05/12/2000 08:55 AM
2. ---------------------- Forwarded by Richard B Sanders/HOU/ECT on 07/09/99 
01:15 PM ---------------------------
   
	
	
	From:  Christi L Nicolay                           07/09/99 12:13 PM
3. Looks good.  Sara

Sara Shackleton
Enron North America Corp.
1400 Smith Street, EB 3801a
Houston, Texas  77002
713-853-5620 (phone)
713-646-3490 (fax)
sara.shackleton@enron.com



	Kimberly Hundl@ENRON
	01/23/2001 02:37 PM
4. ----- Forwarded by Dan J Hyvl/HOU/ECT on 06/12/2001 02:14 PM -----

	sschneider@aep.com
	06/11/2001 11:18 AM
5. ---------------------- Forwarded by Judy Hernandez/HOU/ECT on 03/07/2000 
01:36 PM ---------------------------
   
	Enron North America Corp.
	
	From:  Maria Sandoval                           03/07/2000 12:43 PM
6. ----- Forwarded by Dan J Hyvl/HOU/ECT on 06/12/2001 02:16 PM -----

	Dan J Hyvl
	06/11/2001 11:21 AM
7.

In [ ]:
#blind classification
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",   # or any NLI model
    device=0                            # -1 for CPU, 0 for first GPU
)

labels = ["Study", "Appointment", "Financial", "Business",
          "Commercial", "Fraud", "Trash", "Others"]

def classify_email_zero_shot(subject, body):
    text = f"Subject: {subject}\n\nBody: {body}"
    result = classifier(text, labels, multi_label=False)
    return result["labels"][0]   # top predicted label

# Example
row = subset_df.iloc[0]
pred = classify_email_zero_shot(row["Subject"], row["clean_content"])
print(pred)



In [ ]:
#classify with llm
def classify_email(subject, body):
    prompt = f"""Classify the following email into one of these categories: 
    Study, Appointment, Financial, Business, Commercial, Fraud, Trash, Others.

    Carefully analyze the subject and body of the email and determine the most appropriate category.

    **Email Details:**  
    Subject: {subject}  
    Body: {body}  

    **Category:**"""
    
    device = "cuda:0"
    
  
    inputs = tokenizer(prompt, return_tensors="pt")
    

    max_input_tokens = 512
    input_length = inputs.input_ids.shape[1]

    if input_length > max_input_tokens:

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_input_tokens)


    inputs = inputs.to(device)


    output = model.generate(**inputs, max_new_tokens=50)

    return tokenizer.decode(output[0], skip_special_tokens=True)


subject = subset_df.iloc[0]["Subject"]
body = subset_df.iloc[0]["content"]
category = classify_email(subject, body)
print(f"Email: {subject}\nDự đoán: {category}")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Email: Wind River - Conversion Checklist
Dự đoán: Classify the following email into one of these categories: 
    Study, Appointment, Financial, Business, Commercial, Fraud, Trash, Others.

    Carefully analyze the subject and body of the email and determine the most appropriate category.

    **Email Details:**  
    Subject: Wind River - Conversion Checklist  
    Body: Attached please find an updated draft of the conversion closing checklist.

If you have any questions or if I can be of further assistance, please feel 
free to call me at (713) 220-4427.

S.B.


 - LostCree.DOC  

    **Category:** Commercial

    **Explanation:** This email is about a commercial transaction. It is not a personal email. The subject line mentions the name of a company and the body of the email contains a list of documents. The email is addressed to a person named


In [21]:
subject1 = subset_df.iloc[1]["Subject"]
body1 = subset_df.iloc[1]["content"]
category = classify_email(subject1, body1)
print(f"Email: {subject1}\nDự đoán: {category}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Email: RE: Austin Energy
Dự đoán: Classify the following email into one of these categories: 
    Study, Appointment, Financial, Business, Commercial, Fraud, Trash, Others.

    Carefully analyze the subject and body of the email and determine the most appropriate category.

    **Email Details:**  
    Subject: RE: Austin Energy  
    Body: It may also makes sense to keep the arrangements separate.  The City's claim pursuant to its power deal (presumably with ENA) is likely a general unsecured claim that gives them very little hope of actually collecting.  Sandhill is not a debtor (not yet anyway) and is an assets/commodity that can be freely assigned.  Just a thought.

 -----Original Message-----
From: 	Mann, Kay  
Sent:	Wednesday, December 12, 2001 12:23 PM
To:	Zisman, Stuart
Subject:	FW: Austin Energy



 -----Original Message-----
From: 	Clark, Barton  
Sent:	Wednesday, December 12, 2001 11:42 AM
To:	Keenan, Jeffrey
Cc:	Mann, Kay
Subject:	RE: Austin Energy

For now, Kay and I both